In [86]:
import nltk
from nltk.corpus import stopwords
from collections import defaultdict
import pandas as pd
from senti_classifier import senti_classifier
from gensim import corpora,models
from itertools import chain,islice
from unidecode import unidecode
import json
import operator
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
from collections import Counter
import string
from nltk.stem.lancaster import LancasterStemmer

In [2]:
data=defaultdict(list)
with open("movies.txt") as f:
    subset = list(islice(f,100007))
    for line in subset:
        if line.strip():
            line = line.strip()
            line = line.split(':',1)
            data[line[0]].append(line[1])

In [3]:
dt = pd.DataFrame.from_dict(data,orient='index')
dt = dt.transpose()

In [4]:
def remove_non_ascii(text):
    try:
        return unidecode(unicode(text,encoding="utf-8"))
    except:
        return unidecode(unicode(text,encoding="ISO-8859-1"))

In [5]:
#posScores = []
#negScores = []
#for row in dt["review/text"]:
    #pos_score,neg_score = senti_classifier.polarity_scores(remove_non_ascii(row))
    #posScores.append(pos_score)
    #negScores.append(neg_score)
    
#dt["pos_sentiment"]=posScores
#dt["neg_sentiment"]=negScores


In [6]:
dt['count'] = dt.groupby('product/productId')['product/productId'].transform(pd.Series.value_counts)

In [7]:
pd.DataFrame(dt.sort('count',ascending=False))

/home/josh/.local/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,review/profileName,review/time,product/productId,review/helpfulness,review/summary,review/userId,review/score,review/text,count
2668,Patrick J. Barnes,1215993600,B000063W82,0/0,Old School,A8ZNVZPQGN8C,5.0,Wow this brought back those saturday morning ...,589
2772,"SAFIYA D. HOSKINS ""Author""",1183766400,B000063W82,0/0,Learning is Still Fun!,A2FMRGRTMYQ0DG,4.0,My godchildren love the educational Schoolhou...,589
2756,"S. Adams ""sheriwx""",1190937600,B000063W82,0/0,Love it!,A371HC8OLEOJBE,5.0,Love it love it love it. I don't know how mu...,589
2757,G. Monaco,1190764800,B000063W82,0/0,School House Rock,A3PJNBG1WJFR31,5.0,Must listen and view for children through adu...,589
2758,"Movie Addict ""Addict""",1190332800,B000063W82,0/0,School House Rock,AD82KTVHTQ0E2,5.0,I remember watching this on Saturday morning ...,589
2759,David Allen,1190246400,B000063W82,0/0,SHR Rocks,A9IQA3LSK1KOV,5.0,I was the age group for these growing up. I ...,589
2760,J. Hansen,1190073600,B000063W82,0/0,SCHOOL HOUSE ROCK,A2QQC5KOM40OJ4,5.0,"MY KIDS LOVED THE SCHOOL HOUSE ROCK TAPE, THE...",589
2761,Kindergarten Teacher,1189814400,B000063W82,0/0,School House Rock,A3GMC1BHNMG1NX,5.0,Kids learn the words to songs so easily....wh...,589
2762,"M. Rosie ""M. Rosie""",1187827200,B000063W82,0/0,Brings back memories...and my kids love it too!,A2ZK5ESGWI4TOU,5.0,We sat and watched this DVD for hours...and j...,589
2763,Oasis Mom,1187481600,B000063W82,0/0,School House really does Rock!,A13547L1PPP0OP,5.0,"When I was a kid, School House Rock was a pro...",589


In [8]:
#dt.to_csv("amazon_movies.csv",sep=',')

In [9]:
count = 0
analyzer = SentimentIntensityAnalyzer()
compound = []
rating = []

In [10]:
def analyzeLine(movieText, currentMovie, count, compound, rating):
    keyValuePair = movieText.split(":")
    if(len(keyValuePair) == 2):
        if(keyValuePair[0] == "review/text"):
            vs = analyzer.polarity_scores(keyValuePair[1])
            currentMovie["vs"] = vs

        if(keyValuePair[0] == "product/productId"):
            count += 1
            print("count " + str(count))
            if(len(currentMovie) > 0):

                try:
                        compound.append(currentMovie["vs"]["compound"])
                        rating.append(currentMovie["review/score"])
                        writeToFile("movies2.json", json.dumps(currentMovie))
                except UnicodeDecodeError:
                    print("Oops!  Encoding error while dumping to json")
                    pass
                except KeyError:
                    print("Oops!  Key error while building data set")
                    pass
                except UnicodeEncodeError:
                    print("ENCODER BAD BAD BAD")
                    pass
            currentMovie = {}

        currentMovie[keyValuePair[0]] = keyValuePair[1]

    return count

def writeToFile(fileName, text):
    with open(fileName, "a") as output:
        output.write(text)

def createScatterPlot(x, y):
    plt.scatter(x, y)
    plt.show()

In [11]:
'''
with open("movies.txt") as infile:
    currentMovie = {}
    try:
        for line in infile:
            try:
                print(line)
                count = analyzeLine(line, currentMovie, count, compound, rating)
                if count > 11112:
                #print(json.dumps(movies))
                #writeToFile("movies.json", json.dumps(movies))

                    print("do scatter plot " + str(len(compound) == len(rating)))
                    createScatterPlot(compound,rating)
                    print("done")
                    break
            except UnicodeEncodeError:
                print("ENCODER BAD BAD BAD")
                pass
    except UnicodeDecodeError:
        print("Oops!  Encoding error while reading file")
        pass
    except KeyError:
        print("Oops!  Key error while reading file")
        pass
    except UnicodeEncodeError:
        print("ENCODER BAD BAD BAD")
        pass
'''

'\nwith open("movies.txt") as infile:\n    currentMovie = {}\n    try:\n        for line in infile:\n            try:\n                print(line)\n                count = analyzeLine(line, currentMovie, count, compound, rating)\n                if count > 11112:\n                #print(json.dumps(movies))\n                #writeToFile("movies.json", json.dumps(movies))\n\n                    print("do scatter plot " + str(len(compound) == len(rating)))\n                    createScatterPlot(compound,rating)\n                    print("done")\n                    break\n            except UnicodeEncodeError:\n                print("ENCODER BAD BAD BAD")\n                pass\n    except UnicodeDecodeError:\n        print("Oops!  Encoding error while reading file")\n        pass\n    except KeyError:\n        print("Oops!  Key error while reading file")\n        pass\n    except UnicodeEncodeError:\n        print("ENCODER BAD BAD BAD")\n        pass\n'

In [12]:
def nltk_entities(text):
    """
    Extract entities using the NLTK named entity chunker.
    """
    text = nltk.pos_tag(nltk.word_tokenize(text))
    results = defaultdict(list)
    lst_results = []
    for entity in nltk.ne_chunk(text):
        if isinstance(entity, nltk.tree.Tree):
            etext = " ".join([word for word, tag in entity.leaves()])
            label = entity.label()
        else:
            continue

        if label == 'PERSON':
            key = 'persons'
        elif label == 'ORGANIZATION':
            key = 'organizations'
        elif label == 'LOCATION':
            key = 'locations'
        elif label == 'GPE':
            key = 'other'
        else:
            key = None

        if key:
            results[key].append(etext)
            lst_results.append(etext)

    return lst_results

## Entities

In [13]:
#Most rated movie in dataset is Schoolhouse Rock
schoolHouse = dt[dt['product/productId'].str.contains('B000063W82')]

In [14]:
#Get entities from reviews about Schoolhouse Rock
entities_cntr = Counter()
for txt in schoolHouse['review/text'].values:
    entities_cntr.update(nltk_entities(unidecode(txt.decode('utf-8'))))

sorted_cntr = sorted(entities_cntr.items(), key=operator.itemgetter(1),reverse=True)
print(sorted_cntr[0:10])

[('DVD', 179), ('Schoolhouse Rock', 147), ('School House Rock', 59), ('SHR', 29), ('Great', 26), ('Disney', 23), ('Schoolhouse', 22), ('Lolly', 22), ('ABC', 18), ('Magic Number', 17)]


In [15]:
#2nd most rated movie in dataset is The Last Samurai
lastSamurai = dt[dt['product/productId'].str.contains('B0001G6PZC')]

In [16]:
#Get entities from reviews about The Last Samurai
entities_cntr = Counter()
for txt2 in lastSamurai['review/text'].values:
    entities_cntr.update(nltk_entities(unidecode(txt2.decode('utf-8',errors='replace'))))

sorted_cntrSamurai = sorted(entities_cntr.items(), key=operator.itemgetter(1),reverse=True)
print(sorted_cntrSamurai[0:10])

[('Samurai', 658), ('Japanese', 434), ('Tom Cruise', 424), ('Japan', 410), ('Algren', 271), ('Cruise', 244), ('Katsumoto', 221), ('Ken Watanabe', 151), ('American', 113), ('Wolves', 79)]


In [17]:
#3rd most rated movie in dataset is Saw
saw = dt[dt['product/productId'].str.contains('B000AMWIVM')]

In [18]:
#Get entities from reviews about Saw
entities_cntr = Counter()
for txt3 in saw['review/text'].values:
    entities_cntr.update(nltk_entities(unidecode(txt3.decode('utf-8',errors='replace'))))

sorted_cntrSaw = sorted(entities_cntr.items(), key=operator.itemgetter(1),reverse=True)
print(sorted_cntrSaw[0:10])

[('Saw', 210), ('Jigsaw', 152), ('Adam', 146), ('Danny Glover', 112), ('Cary Elwes', 94), ('SAW', 88), ('Leigh Whannell', 65), ('Elwes', 63), ('Lawrence', 59), ('Gordon', 58)]


In [19]:
#4th most rated movie in dataset is Super8
super8 = dt[dt['product/productId'].str.contains('B004EPYZQ2')]

In [20]:
#Get entities from reviews about Super 8
entities_cntr = Counter()
for txt4 in super8['review/text'].values:
    entities_cntr.update(nltk_entities(unidecode(txt4.decode('utf-8',errors='replace'))))

sorted_cntrSuper = sorted(entities_cntr.items(), key=operator.itemgetter(1),reverse=True)
print(sorted_cntrSuper[0:10])

[('Abrams', 183), ('Spielberg', 158), ('Joe', 152), ('Steven Spielberg', 67), ('Goonies', 58), ('Elle Fanning', 54), ('Charles', 53), ('Joel Courtney', 49), ('ET', 39), ('Alice', 37)]


# Topic Modeling Schoolhouse Rock

In [21]:
exclude = set(string.punctuation)
tokenized = []
for doc in schoolHouse['review/text']:
    tokens = nltk.word_tokenize(doc.lower())
    tokenized.append(''.join([ch for ch in ' '.join(tokens) if ch not in exclude]).split())

In [22]:
#Remove Stopwords
sws = set(stopwords.words('english'))
sws.add('DVD')
sws.add('dvd')
sws_removed = []
for j,sent in enumerate(tokenized):
    sws_removed.append([i for i in sent if i not in sws])

In [23]:
dictionary = corpora.Dictionary(sws_removed)

In [24]:
corpus = [dictionary.doc2bow(text) for text in sws_removed]

In [25]:
ldamodel = models.ldamodel.LdaModel(corpus, num_topics=3, id2word = dictionary, passes=20)

In [26]:
print(ldamodel.print_topics(num_topics=3, num_words=3))

[(0, u'0.018*"songs" + 0.016*"great" + 0.016*"rock"'), (1, u'0.032*"br" + 0.013*"rock" + 0.012*"school"'), (2, u'0.015*"rock" + 0.014*"br" + 0.010*"songs"')]


In [27]:
ldamodel.get_document_topics(corpus[0])

[(1, 0.98477476947101994)]

## Stemming

In [28]:
st = LancasterStemmer()
stemmedWords = []

sws = set(stopwords.words('english'))
sws.add('DVD')
sws.add('dvd')
sws_removed = []
for j,sent in enumerate(tokenized):
    sws_removed.append([st.stem(i) for i in sent if i not in sws])  

In [29]:
dictionary = corpora.Dictionary(sws_removed)

In [30]:
corpus = [dictionary.doc2bow(text) for text in sws_removed]

In [31]:
ldamodel = models.ldamodel.LdaModel(corpus, num_topics=3, id2word = dictionary, passes=20)

In [32]:
print(ldamodel.print_topics(num_topics=3, num_words=3))

[(0, u'0.024*"br" + 0.015*"rock" + 0.013*"us"'), (1, u'0.019*"br" + 0.008*"kid" + 0.008*"back"'), (2, u'0.023*"song" + 0.019*"lov" + 0.018*"rock"')]


In [33]:
ldamodel.show_topics()

[(0,
  u'0.024*"br" + 0.015*"rock" + 0.013*"us" + 0.010*"schoolhouse" + 0.009*"school" + 0.009*"teach" + 0.008*"stud" + 0.008*"kid" + 0.007*"video" + 0.007*"learn"'),
 (1,
  u'0.019*"br" + 0.008*"kid" + 0.008*"back" + 0.007*"rock" + 0.007*"mem" + 0.007*"watch" + 0.007*"cartoon" + 0.007*"morn" + 0.007*"saturday" + 0.007*"lik"'),
 (2,
  u'0.023*"song" + 0.019*"lov" + 0.018*"rock" + 0.018*"learn" + 0.017*"watch" + 0.016*"kid" + 0.013*"br" + 0.013*"gre" + 0.010*"schoolhouse" + 0.010*"rememb"')]

In [34]:
print ldamodel

LdaModel(num_terms=2406, num_topics=3, decay=0.5, chunksize=2000)


## Topic Documents

In [35]:
def cleanup_characters(text):
    newArray = []
    for txt in text:
        newArray.append(remove_non_ascii(txt))
        
    return newArray

In [39]:
lda_corpus = ldamodel[corpus]

scores = list(chain(*[[score for topic_id,score in topic] 
                      for topic in [doc for doc in lda_corpus]]))

threshold = sum(scores)/len(scores)

cluster1 = [j for i,j in zip(lda_corpus,schoolHouse['review/text']) if i[0][1] > threshold]
#cluster2 = [j for i,j in zip(lda_corpus,schoolHouse['review/text']) if i[1][1] > threshold]
#cluster3 = [j for i,j in zip(lda_corpus,schoolHouse['review/text']) if i[2][1] > threshold]

In [40]:
cluster1_cleaned = cleanup_characters(cluster1)

pos_score, neg_score = senti_classifier.polarity_scores(cluster1_cleaned)
print "Positive Score:",pos_score,"Negative Score:", neg_score

Positive Score: 637.875 Negative Score: 400.75


## Topic Modeling The Last Samurai

In [46]:
exclude = set(string.punctuation)
tokenized = []
for doc in lastSamurai['review/text']:
    doc = doc.decode('utf-8',errors='replace')
    tokens = nltk.word_tokenize(doc.lower())
    tokenized.append(''.join([ch for ch in ' '.join(tokens) if ch not in exclude]).split())

In [47]:
#Remove Stopwords
sws = set(stopwords.words('english'))
sws.add('DVD')
sws.add('dvd')
sws_removed = []
for j,sent in enumerate(tokenized):
    sws_removed.append([i for i in sent if i not in sws])

In [48]:
dictionary = corpora.Dictionary(sws_removed)

In [49]:
corpus = [dictionary.doc2bow(text) for text in sws_removed]

In [50]:
ldamodel = models.ldamodel.LdaModel(corpus, num_topics=3, id2word = dictionary, passes=20)

In [51]:
print(ldamodel.print_topics(num_topics=3, num_words=3))

[(0, u'0.023*"samurai" + 0.020*"movie" + 0.016*"br"'), (1, u'0.034*"br" + 0.014*"movie" + 0.013*"samurai"'), (2, u'0.008*"film" + 0.004*"p" + 0.004*"one"')]


In [52]:
ldamodel.get_document_topics(corpus[0])

[(0, 0.20511451366052283), (1, 0.79389954489343029)]

## Stemming

In [53]:
st = LancasterStemmer()
stemmedWords = []

sws = set(stopwords.words('english'))
sws.add('DVD')
sws.add('dvd')
sws_removed = []
for j,sent in enumerate(tokenized):
    sws_removed.append([st.stem(i) for i in sent if i not in sws])  

In [54]:
dictionary = corpora.Dictionary(sws_removed)

In [55]:
corpus = [dictionary.doc2bow(text) for text in sws_removed]

In [56]:
ldamodel = models.ldamodel.LdaModel(corpus, num_topics=3, id2word = dictionary, passes=20)

In [57]:
print(ldamodel.print_topics(num_topics=3, num_words=3))

[(0, u'0.045*"movy" + 0.014*"samura" + 0.014*"cru"'), (1, u'0.029*"samura" + 0.028*"br" + 0.014*"film"'), (2, u'0.018*"br" + 0.017*"movy" + 0.017*"film"')]


In [58]:
ldamodel.show_topics()

[(0,
  u'0.045*"movy" + 0.014*"samura" + 0.014*"cru" + 0.013*"act" + 0.011*"tom" + 0.009*"nt" + 0.009*"gre" + 0.009*"film" + 0.008*"see" + 0.008*"on"'),
 (1,
  u'0.029*"samura" + 0.028*"br" + 0.014*"film" + 0.013*"movy" + 0.011*"jap" + 0.009*"japanes" + 0.009*"cru" + 0.007*"last" + 0.007*"tom" + 0.007*"algr"'),
 (2,
  u'0.018*"br" + 0.017*"movy" + 0.017*"film" + 0.014*"samura" + 0.013*"cru" + 0.010*"act" + 0.009*"p" + 0.008*"algr" + 0.008*"on" + 0.007*"nt"')]

In [59]:
print ldamodel

LdaModel(num_terms=5575, num_topics=3, decay=0.5, chunksize=2000)


## Topic Documents

In [62]:
lda_corpus = ldamodel[corpus]

scores = list(chain(*[[score for topic_id,score in topic] 
                      for topic in [doc for doc in lda_corpus]]))

threshold = sum(scores)/len(scores)

cluster1 = [j for i,j in zip(lda_corpus,lastSamurai['review/text']) if i[0][1] > threshold]
#cluster2 = [j for i,j in zip(lda_corpus,lastSamurai['review/text']) if i[1][1] > threshold]
#cluster3 = [j for i,j in zip(lda_corpus,lastSamurai['review/text']) if i[2][1] > threshold]

In [64]:
#Positive: 2406.944 Negative: 1246.125

In [63]:
cluster1_cleaned = cleanup_characters(cluster1)

pos_score, neg_score = senti_classifier.polarity_scores(cluster1_cleaned)
print "Positive Score:",pos_score,"Negative Score:", neg_score

Positive Score: 2406.944 Negative Score: 1246.125


## Topic Modeling Saw

In [69]:
exclude = set(string.punctuation)
tokenized = []
for doc in saw ['review/text']:
    doc = doc.decode('utf-8',errors='replace')
    tokens = nltk.word_tokenize(doc.lower())
    tokenized.append(''.join([ch for ch in ' '.join(tokens) if ch not in exclude]).split())

In [70]:
#Remove Stopwords
sws = set(stopwords.words('english'))
sws.add('DVD')
sws.add('dvd')
sws_removed = []
for j,sent in enumerate(tokenized):
    sws_removed.append([i for i in sent if i not in sws])

In [71]:
dictionary = corpora.Dictionary(sws_removed)

In [72]:
corpus = [dictionary.doc2bow(text) for text in sws_removed]

In [73]:
ldamodel = models.ldamodel.LdaModel(corpus, num_topics=3, id2word = dictionary, passes=20)

In [74]:
print(ldamodel.print_topics(num_topics=3, num_words=3))

[(0, u'0.012*"br" + 0.006*"saw" + 0.004*"que"'), (1, u'0.006*"film" + 0.005*"gordon" + 0.004*"jigsaw"'), (2, u'0.034*"br" + 0.023*"movie" + 0.014*"saw"')]


## Stemming

In [84]:
st = LancasterStemmer()
stemmedWords = []

sws = set(stopwords.words('english'))
sws.add('DVD')
sws.add('dvd')
sws_removed = []
for j,sent in enumerate(tokenized):
    sws_removed.append([st.stem(i) for i in sent if i not in sws])  

In [76]:
dictionary = corpora.Dictionary(sws_removed)

In [77]:
corpus = [dictionary.doc2bow(text) for text in sws_removed]

In [78]:
ldamodel = models.ldamodel.LdaModel(corpus, num_topics=3, id2word = dictionary, passes=20)

In [79]:
print(ldamodel.print_topics(num_topics=3, num_words=3))

[(0, u'0.011*"br" + 0.006*"que" + 0.005*"de"'), (1, u'0.029*"br" + 0.014*"saw" + 0.014*"kil"'), (2, u'0.034*"br" + 0.034*"movy" + 0.016*"film"')]


In [80]:
ldamodel.show_topics()

[(0,
  u'0.011*"br" + 0.006*"que" + 0.005*"de" + 0.005*"un" + 0.005*"la" + 0.004*"sev" + 0.004*"el" + 0.003*"es" + 0.003*"pelicul" + 0.003*"saw"'),
 (1,
  u'0.029*"br" + 0.014*"saw" + 0.014*"kil" + 0.014*"film" + 0.010*"movy" + 0.008*"adam" + 0.008*"on" + 0.007*"hor" + 0.007*"jigsaw" + 0.007*"two"'),
 (2,
  u'0.034*"br" + 0.034*"movy" + 0.016*"film" + 0.015*"nt" + 0.014*"saw" + 0.012*"act" + 0.010*"lik" + 0.010*"hor" + 0.010*"on" + 0.009*"real"')]

In [81]:
print ldamodel

LdaModel(num_terms=4990, num_topics=3, decay=0.5, chunksize=2000)


## Topic Documents

In [82]:
lda_corpus = ldamodel[corpus]

scores = list(chain(*[[score for topic_id,score in topic] 
                      for topic in [doc for doc in lda_corpus]]))

threshold = sum(scores)/len(scores)

cluster1 = [j for i,j in zip(lda_corpus,saw['review/text']) if i[0][1] > threshold]
#cluster2 = [j for i,j in zip(lda_corpus,lastSamurai['review/text']) if i[1][1] > threshold]
#cluster3 = [j for i,j in zip(lda_corpus,lastSamurai['review/text']) if i[2][1] > threshold]

In [83]:
cluster1_cleaned = cleanup_characters(cluster1)

pos_score, neg_score = senti_classifier.polarity_scores(cluster1_cleaned)
print "Positive Score:",pos_score,"Negative Score:", neg_score

Positive Score: 1647.661 Negative Score: 1518.904


### Positive Score: 1647.661 Negative Score: 1518.904

## Topic Modeling Super 8

In [87]:
exclude = set(string.punctuation)
tokenized = []
for doc in super8['review/text']:
    doc = doc.decode('utf-8',errors='replace')
    tokens = nltk.word_tokenize(doc.lower())
    tokenized.append(''.join([ch for ch in ' '.join(tokens) if ch not in exclude]).split())

In [88]:
#Remove Stopwords
sws = set(stopwords.words('english'))
sws.add('DVD')
sws.add('dvd')
sws_removed = []
for j,sent in enumerate(tokenized):
    sws_removed.append([i for i in sent if i not in sws])

In [89]:
dictionary = corpora.Dictionary(sws_removed)

In [90]:
corpus = [dictionary.doc2bow(text) for text in sws_removed]

In [91]:
ldamodel = models.ldamodel.LdaModel(corpus, num_topics=3, id2word = dictionary, passes=20)

In [92]:
print(ldamodel.print_topics(num_topics=3, num_words=3))

[(0, u'0.012*"br" + 0.005*"movie" + 0.005*"alien"'), (1, u'0.040*"br" + 0.018*"film" + 0.012*"movie"'), (2, u'0.035*"movie" + 0.018*"br" + 0.013*"nt"')]


## Stemming

In [93]:
st = LancasterStemmer()
stemmedWords = []

sws = set(stopwords.words('english'))
sws.add('DVD')
sws.add('dvd')
sws_removed = []
for j,sent in enumerate(tokenized):
    sws_removed.append([st.stem(i) for i in sent if i not in sws])  

In [94]:
dictionary = corpora.Dictionary(sws_removed)

In [95]:
corpus = [dictionary.doc2bow(text) for text in sws_removed]

In [96]:
ldamodel = models.ldamodel.LdaModel(corpus, num_topics=3, id2word = dictionary, passes=20)

In [97]:
print(ldamodel.print_topics(num_topics=3, num_words=3))

[(0, u'0.024*"br" + 0.017*"aly" + 0.010*"movy"'), (1, u'0.029*"br" + 0.025*"film" + 0.012*"sup"'), (2, u'0.044*"movy" + 0.031*"br" + 0.014*"film"')]


In [98]:
ldamodel.show_topics()

[(0,
  u'0.024*"br" + 0.017*"aly" + 0.010*"movy" + 0.010*"film" + 0.009*"lik" + 0.008*"kid" + 0.007*"et" + 0.006*"spielberg" + 0.006*"on" + 0.006*"nt"'),
 (1,
  u'0.029*"br" + 0.025*"film" + 0.012*"sup" + 0.012*"8" + 0.010*"movy" + 0.009*"joe" + 0.009*"abram" + 0.008*"spielberg" + 0.008*"train" + 0.008*"kid"'),
 (2,
  u'0.044*"movy" + 0.031*"br" + 0.014*"film" + 0.014*"nt" + 0.013*"kid" + 0.012*"act" + 0.011*"lik" + 0.010*"real" + 0.010*"good" + 0.008*"watch"')]

In [99]:
print ldamodel

LdaModel(num_terms=4541, num_topics=3, decay=0.5, chunksize=2000)


## Topic Documents

In [100]:
lda_corpus = ldamodel[corpus]

scores = list(chain(*[[score for topic_id,score in topic] 
                      for topic in [doc for doc in lda_corpus]]))

threshold = sum(scores)/len(scores)

cluster1 = [j for i,j in zip(lda_corpus,super8['review/text']) if i[0][1] > threshold]
#cluster2 = [j for i,j in zip(lda_corpus,lastSamurai['review/text']) if i[1][1] > threshold]
#cluster3 = [j for i,j in zip(lda_corpus,lastSamurai['review/text']) if i[2][1] > threshold]

In [101]:
cluster1_cleaned = cleanup_characters(cluster1)

pos_score, neg_score = senti_classifier.polarity_scores(cluster1_cleaned)
print "Positive Score:",pos_score,"Negative Score:", neg_score

Positive Score: 1681.51 Negative Score: 1050.07


## Positive Score: 1681.51 Negative Score: 1050.07